In [1]:
from model.layers.ResidualPool import *
import mxnet
import numpy as np
from mxnet import nd, autograd
from model.layers.AttentionPartsCRF import *

def transformer(data, label):
    data = mx.image.imresize(data, 224, 224)
    data = mx.nd.transpose(data, (2,0,1))
    data = data.astype(np.float32)
    return data, label

ctx = mx.gpu()
batch_size = 1
train_data = mx.gluon.data.DataLoader(mxnet.gluon.data.vision.ImageFolderDataset
                                      ("/home/dan/test_img",transform=transformer),batch_size=1)

for d, l in train_data:
    break


#et = ResidualPool(64)
from model.layers.AttentionPartsCRF import *
from model.hg_attention import *
#net = AttentionPartsCRF(1,4,1)
net = createModel( 4, 8, 256,64,1, )

net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)


/home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring /home/dan/test_img/ff0025574be642b231354d49556304bd39a9b88a.jpg, which is not a directory.
  app.launch_new_instance()
/home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring /home/dan/test_img/keypoint_train_annotations_20170909.json, which is not a directory.
  app.launch_new_instance()
/home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring /home/dan/test_img/train.idx, which is not a directory.
  app.launch_new_instance()
/home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring /home/dan/test_img/train.lst, which is not a directory.
  app.launch_new_instance()
/home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Ignoring /home/dan/test_img/train.rec, which is not a director

In [3]:

trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': .01})


In [4]:
net.collect_params()

createmodel1_ (
  Parameter createmodel1_conv0_weight (shape=(64, 0, 7, 7), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_conv0_bias (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_batchnorm0_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_batchnorm0_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_batchnorm0_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_batchnorm0_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_residual0_convblock0_batchnorm0_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_residual0_convblock0_batchnorm0_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_residual0_convblock0_batchnorm0_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter createmodel1_residual0_convblock0_batchnorm0_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter crea

In [5]:

softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()


def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for d, l in data_iterator:
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
    return acc.get()[1]

def train(net):
    epochs = 10
    smoothing_constant = .01

    for e in range(epochs):
        for i, (d, l) in enumerate(train_data):
            data = d.as_in_context(ctx)
            label = l.as_in_context(ctx)
            with autograd.record():
                output = net(data)
                loss = softmax_cross_entropy(output, label)

            loss.backward()
            trainer.step(data.shape[0])

            ##########################
            #  Keep a moving average of the losses
            ##########################
            curr_loss = nd.mean(loss).asscalar()

        train_accuracy = evaluate_accuracy(train_data, net)
        moving_loss = (curr_loss if ((i == 0) and (e == 0)) else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)
        print("Epoch %s. Loss: %s, Train_acc %s" % (e, moving_loss, train_accuracy))

train(net)

infer_shape error. Arguments:
  input_0: (1, 5)


MXNetError: Error in operator createmodel1_hourglass0_represidual0_residualpool1_convblock0_conv0_fwd: [15:21:26] src/operator/./convolution-inl.h:491: Check failed: dshp.ndim() == 4U (2 vs. 4) Input data should be 4D in batch-num_filter-y-x

Stack trace returned 10 entries:
[bt] (0) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x1f7a8c) [0x7f36bff06a8c]
[bt] (1) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x11bdcb0) [0x7f36c0ecccb0]
[bt] (2) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0x113aff7) [0x7f36c0e49ff7]
[bt] (3) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xf7fae7) [0x7f36c0c8eae7]
[bt] (4) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(+0xf817cb) [0x7f36c0c907cb]
[bt] (5) /home/dan/.conda/envs/tensorflow/lib/python3.6/site-packages/mxnet/libmxnet.so(MXSymbolInferShape+0x17ed) [0x7f36c0c2799d]
[bt] (6) /

In [ ]:
height = 254
kernel_size = 4
padding = np.ceil(kernel_size / 2) -0.5
dilation = 1
stride = 1
np.floor((height+2*padding-dilation*(kernel_size-1)-1)/stride)+1

In [4]:
with open("001.jpg", 'rb') as fp:
    str_image = fp.read()

In [5]:
image = mx.image.imdecode(str_image)

In [17]:
image.shape

(426, 640, 3)

In [18]:
cropped_image = mx.image.fixed_crop(src=image,x0=0,y0=0, 
                                    w=image.shape[1],
                                    h=image.shape[0],
                                    size=(256,256))




(256, 256, 3)

In [23]:
cp = cropped_image.asnumpy()

In [ ]:
import mxnet as mx
data_iter = mx.io.ImageDetRecordIter(
  path_imgrec="/home/dan/test_img/train.rec", # The target record file.
  # Output data shape; 227x227 region will be cropped from the original image.
  data_shape=(3, 256, 256),
    batch_size=1, # Number of items per batch.

  # You can specify more augmentation options. Use help(mx.io.ImageRecordIter) to see all the options.
  )

In [1]:
import mxnet as mx

In [ ]:
bn1 = mx.symbol.BatchNorm()